### Задание 1

Постройте иерархию наследования для какого-нибудь набора объектов с нетривиальной структурой. Например, 
используйте для этого биологическую систематику живых организмов (см. Википедию); спуститесь в ней от класса 
«Живое» до конкретных видов. 

Полная систематика слишком большая, поэтому большую часть уровней и веток можно пропустить.

In [5]:
from abc import ABC, abstractmethod


class Animal(ABC):
    '''Животное'''

    def __init__(self, size, weight):
        self._size = size
        self._weight = weight

    @property
    @abstractmethod
    def is_angry(self):
        pass

    def get_size(self):
        return self._size

    def get_weight(self):
        return self._weight


class Chordate(Animal):
    '''Хордовое'''
    pass


class Mammal(Chordate):
    '''Млекопитающие'''
    pass


class Carnivora(Mammal):
    '''Хищник'''
    pass


class Canidae(Carnivora):
    '''Псовые'''
    pass


class Canis(Canidae):
    '''Волки'''
    pass


class Wolf(Canis):
    '''Волк'''

    def is_angry(self):
        return True


class Dog(Wolf):
    '''Собака'''

    def is_angry(self):
        return False


class GoldenRetriever(Dog):
    '''Золотистый ретривер'''

    def has_gold_furr(self):
        return True

In [6]:
wolf = Wolf(4, 12)
print(wolf.get_size())
print(wolf.get_weight())
print(wolf.is_angry())

print('\n')

doge = GoldenRetriever(4, 10)
print(doge.get_size())
print(doge.get_weight())
print(doge.is_angry())
print(doge.has_gold_furr())

4
12
True


4
10
False
True


### Задание 2

Реализуйте класс «n-мерный» вектор. У этого класса должны быть определены все естественные для вектора операции:
- сложение
- вычитание
- умножение на константу
- скалярное произведение
- сравнение на равенство

а также операции 
- вычисления длины
- получение элемента по индексу 
- и строковое представление. 

Во всех операциях можно считать, что все передаваемые аргументы корректны.

In [7]:
class Vector:
    def __init__(self, *coords):
        self._coordinates = list(coords)

    def __len__(self):
        return len(self._get_coords())

    def __add__(self, vector):
        self._check_type(vector)
        self._compare_lengths(vector)

        self_coords = self._get_coords()
        vector_coodrs = vector._get_coords()

        new_coords = []
        for self_elem, vector_elem in zip(self_coords, vector_coodrs):
            new_coords.append(self_elem + vector_elem)

        return Vector(*new_coords)

    def __sub__(self, vector):
        self._check_type(vector)
        self._compare_lengths(vector)

        self_coords = self._get_coords()
        vector_coodrs = vector._get_coords()

        new_coords = []
        for self_elem, vector_elem in zip(self_coords, vector_coodrs):
            new_coords.append(self_elem - vector_elem)

        return Vector(*new_coords)

    def __mul__(self, const):
        if not isinstance(const, int) and not isinstance(const, float):
            raise TypeError('Argument type is not a number')

        self_coords = self._get_coords()

        new_coords = []
        for self_elem in self_coords:
            new_coords.append(self_elem * const)

        return Vector(*new_coords)

    def dot(self, vector):
        self._check_type(vector)
        self._compare_lengths(vector)

        self_coords = self._get_coords()
        vector_coodrs = vector._get_coords()

        return sum([self_elem * vector_elem for self_elem,
                    vector_elem in zip(self_coords, vector_coodrs)])

    def __eq__(self, vector):
        if not isinstance(vector, Vector):
            return False

        self_coords = self._get_coords()
        vector_coodrs = vector._get_coords()

        if len(self) != len(vector):
            return False

        for self_elem, vector_elem in zip(self_coords, vector_coodrs):
            if self_elem != vector_elem:  # по хорошему float нужно сравнивать по модулю
                return False

        return True

    def __getitem__(self, index):
        if not isinstance(index, int):
            raise TypeError('Index type is not an int')

        if index >= len(self):
            raise ValueError('Out of range')

        return self._get_coords()[index]

    def __str__(self):
        return ' '.join(map(str, self._get_coords()))

    def _get_coords(self):
        return self._coordinates

    def _check_type(self, arg):
        if not isinstance(arg, Vector):
            raise TypeError('Argument type is not Vector')

    def _compare_lengths(self, vector):
        if len(self) != len(vector):
            raise TypeError('Vectors have different lengths')

In [8]:
a = Vector(2, 32, 21.32, 32)
b = Vector(52, 23, 12.55, 87)
c = Vector(2, 32, 21.32, 32)

print(str(a + b))
print(str(a - b))
print(str(a * 4))
print(a.dot(b))
print(a == b)
print(a == c)
print(b[3])

54 55 33.870000000000005 119
-50 9 8.77 -55
8 128 85.28 128
3891.566
False
True
87


### Задание 3

Дано некоторое множество объектов X и натуральное число n. Необходимо перебрать все элементы в множестве X^n (декартово произведение). 

Например, если $X = \{1, a\}$, то $X^2$ содержит такие элементы: (1, 1), (1, a), (a, 1), (a, a).

Для этого напишите класс, экземпляры которого соответствую элементам этого множества, и у которых есть методы для получения текущего элемента, и замены текущего элемента на «следующий». 

Следующий элемент выбирайте таким образом, чтобы при последовательном применении этого метода можно было получить все элементы множества. Не забудьте добавить возможность зацикливания, если этот метод вызывается более $|X|^n$ раз.

_Примечание_. Для определенности считайте, что множество $X$ задается списком (то есть на элементах есть порядок), но в нем все элементы уникальны.

In [9]:
class CartesianProduct:
    def __init__(self, user_set, power, start=0):
        self._set = user_set
        self._power = power
        self._counter = start
        self._mod_number = len(user_set)**power

    def next(self):
        return CartesianProduct(self._set, self._power, self._counter + 1)

    def __str__(self):
        result_in_numbers = self._to_number_system(self._counter %
                                                   self._mod_number, len(self._set))

        to_print = self._get_in_symbols(result_in_numbers)

        return ', '.join(map(str, to_print))

    def _get_in_symbols(self, numbers):
        result = []

        if self._power > len(numbers):
            numbers = self._pad_list(numbers, self._power)

        for number in numbers:
            result.append(self._set[number])

        return result

    def _pad_list(self, list_to_add, pad):
        zeros_to_add = pad - len(list_to_add)
        pad_list = [0] * zeros_to_add

        return pad_list + list_to_add

    def _to_number_system(self, number, system):
        result = []

        if number == 0:
            return [0]

        while number >= system - 1:
            result.append(number % system)
            number //= system

        if number > 0:
            result.append(number)

        result.reverse()

        return result

In [10]:
cur = CartesianProduct([1, 'a', 'b'], 3)

for i in range(30):
    print(cur)
    cur = cur.next()

1, 1, 1
1, 1, a
1, 1, b
1, a, 1
1, a, a
1, a, b
1, b, 1
1, b, a
1, b, b
a, 1, 1
a, 1, a
a, 1, b
a, a, 1
a, a, a
a, a, b
a, b, 1
a, b, a
a, b, b
b, 1, 1
b, 1, a
b, 1, b
b, a, 1
b, a, a
b, a, b
b, b, 1
b, b, a
b, b, b
1, 1, 1
1, 1, a
1, 1, b
